In [2]:
import logging 
import pandas as pd
from pathlib import Path
from glob import glob 
import os
from tqdm import tqdm 
import sqlite3
import argparse
import json
from json.decoder import JSONDecodeError
import sys
from event_ids import iosKeys, andKeys
from pprint import pprint
import ast
import re


In [2]:
path = os.path.abspath('/opt/data/multicast/phone')
iosSenses = glob((path)+'/*/*.db')
andSenses = glob((path)+'/*/*.dbr')
to_analyze =  iosSenses + andSenses



In [67]:

#andKeys = pd.DataFrame(list(andKeys.items()), columns=['label', 'Key'])
error_files = set()
c= 0 
df = pd.DataFrame()

ios_map_df = pd.DataFrame(list(iosKeys.items()), columns=['label', 'Key'])
and_map_df = pd.DataFrame(list(andKeys.items()), columns=['label', 'Key'])



def decode_json(x):
    if isinstance(x, bytes):
        try:
            return json.loads(x.decode())
        except json.JSONDecodeError: 
            return x.decode()
    else:
        return x

for dbloc in tqdm(to_analyze):
    conn = sqlite3.connect(dbloc)
    try:
        dft = pd.read_sql(sql="SELECT * FROM events", con=conn)
    except pd.errors.DatabaseError:
        ex_type, ex_value, ex_traceback = sys.exc_info()
        if str(ex_value) == """Execution failed on sql 'SELECT * FROM events': no such table: events""":
            pass
        else:
            logging.error(f"Can't read database file: {dbloc}. --- error: {ex_value} ")
            error_files.add(dbloc)
        continue

    if dbloc[-3:] == 'dbr':
        dft = dft.merge(and_map_df,how='left',left_on='data_type',right_on='Key')
        dft = dft[['timestamp','data_type','data','label']]
        dft.rename(columns={
                            'data_type' : 'event_id'
                                            }, inplace = True )
        dft['data'] = dft['data'].map(decode_json)  
        dft['user_id'] = dbloc.split(os.sep)[-1].split('_')[1].split('.')[0] #get a portion of the folder name 
        dft['os'] = 'android'
        
    elif dbloc[-3:] == '.db':
        dft = dft.merge(ios_map_df,how='left',left_on='event_id',right_on='Key')
        dft = dft[['event_time','event_id','event_data','label','device_id', 'user_id']]
        dft['event_data'] = dft['event_data'].map(decode_json) 
        dft['os'] = 'ios'
        dft.rename(columns={
                            'event_data' : 'data',
                            'event_time' : 'timestamp'
                                            }, inplace = True )


    dft['path'] = dbloc
    df = pd.concat([df,dft]) #TODO launch parse 
    c+=1

        
    conn.close()
    if c==1000:
        break


  9%|▊         | 2600/30506 [04:08<44:26, 10.47it/s]  


In [68]:
dfios = df.loc[df['os'] == 'ios'].drop_duplicates(subset = 'event_id', keep='last')
df.drop_duplicates(subset = 'event_id', keep='last')

,timestamp,data_type,data,label,user_id,os,path,device_id
1457,1.696358e+09,202,"{'start_time': 1696356911451, 'end_time': 1696...",EVENT_STEP_COUNT,1696321890820_651897c0b573c8000fcd559e.dbr,android,/opt/data/multicast/phone/651897c0b573c8000fcd...,NaN
594,1.698318e+09,10,"{'DEVICES': [{'CLASS': '1f00', 'com.samsung.bl...",EVENT_BLUETOOTH,1698313966969_65391c7c55b3c800135fd2d2.dbr,android,/opt/data/multicast/phone/65391c7c55b3c800135f...,NaN
404,1.702026e+09,210,[{'number': '83653d9d0e8628eb301cef41df5722502...,EVENT_CALL_LOG,1701965139149_65391c7c55b3c800135fd2d2.dbr,android,/opt/data/multicast/phone/65391c7c55b3c800135f...,NaN
513,1.699086e+09,211,[{'address': 'a41b5e08e76671e7507e7e83e69d7130...,EVENT_SMS_LOG,1699045906840_65391c7c55b3c800135fd2d2.dbr,android,/opt/data/multicast/phone/65391c7c55b3c800135f...,NaN
706,1.699090e+09,91,{'SSID': '3310c74974594ea4367dfbb7ad0bf5473a59...,EVENT_WIFI_CONNECTED,1699045906840_65391c7c55b3c800135fd2d2.dbr,android,/opt/data/multicast/phone/65391c7c55b3c800135f...,NaN
710,1.699090e+09,9,{'WIFIDATA': []},EVENT_WIFI,1699045906840_65391c7c55b3c800135fd2d2.dbr,android,/opt/data/multicast/phone/65391c7c55b3c800135f...,NaN
189,1.699687e+09,2,"{'HASALTITUDE': True, 'ALTITUDE': 939.30004882...",EVENT_LOCATION,1699662051794_65391c7c55b3c800135fd2d2.dbr,android,/opt/data/multicast/phone/65391c7c55b3c800135f...,NaN
551,1.699749e+09,199,{'running_services': ['com.varunmishra.pssenso...,EVENT_RUNNING_SERVICES,1699662051794_65391c7c55b3c800135fd2d2.dbr,android,/opt/data/multicast/phone/65391c7c55b3c800135f...,NaN
553,1.699749e+09,902,1,EVENT_LOCATION_PING,1699662051794_65391c7c55b3c800135fd2d2.dbr,android,/opt/data/multicast/phone/65391c7c55b3c800135f...,NaN
554,1.699749e+09,136,{'screen_state': 1},EVENT_SCREEN_EVENT,1699662051794_65391c7c55b3c800135fd2d2.dbr,android,/opt/data/multicast/phone/65391c7c55b3c800135f...,NaN


## pezzi

In [ ]:
iosKeys = {
    'EVENT_ID_BATTERY':  11, #is this event based or follows a sampling schedule? 
    'EVENT_ID_BATTERY_STATE':  111, #is this event based or follows a sampling schedule?

    'EVENT_ID_LOCATION_ALL':  151,
    'EVENT_ID_LOCATION_APP':  152,

    'EVENT_ID_WIFI':  18,
    'EVENT_ID_WIFI_STATE':  181, #is this event based or follows a sampling schedule? does it always co occur with wifi rec?

    'EVENT_ID_BLUETOOTH':  19, 
    'EVENT_ID_STEP':  21,
    'EVENT_ID_HEALTH_KIT':  22,
    'EVENT_ID_CALL_LOG':  23,
    'EVENT_ID_HEART_BEAT':  987, 
    'EVENT_ID_ACTIVITY':  16,
    'EVENT_ID_BRIGHTNESS':  13,
    'EVENT_ID_LOCK_UNLOCK':  14,

    'EVENT_ID_ACCELEROMETER':  20, #never once seen LOL
    'EVENT_ID_AUDIO_INFERENCE':  24, #never once seen LOL
    'EVENT_ID_MOTION':  17, #never once seen LOL
    'EVENT_ID_CONVO':  241, #never once seen LOL
    'EVENT_ID_LOCATION_JS':  15, #never once seen LOL

}



In [1]:
import pandas as pd 
andKeys = {
    'EVENT_BATTERY_LEVEL':  171,

    'EVENT_LOCATION':  2,
    'EVENT_LOCATION_PING':  902,

    'EVENT_WIFI':  9,
    'EVENT_WIFI_CONNECTED':  91,

    'EVENT_BLUETOOTH':  10,
    'EVENT_STEP_COUNT':  202,

    'EVENT_CALL_LOG':  210,
    'EVENT_SCREEN_EVENT':  136,

    'EVENT_SMS_LOG':  211, #si riparte da qui, e questi sono esclusivi per android
    'EVENT_APP_USAGE':  22,
    'EVENT_SERVICES_STARTED':  11,
    'EVENT_RUNNING_SERVICES':  199,
    'EVENT_NOTIFICATION':  301,

#non pervenuti
    'EVENT_LIGHT_SENSOR_READING':  141,
    'EVENT_DARK_PERIOD':  14,
    'EVENT_PHONE_LOCK':  100,
    'EVENT_SCREEN_OFF_PERIOD':  15,
    'EVENT_PHONE_BOOTUP':  16,
    'EVENT_AUDIO_LEVEL':  110,
    'EVENT_AUDIO_FEAT':  111,
    'EVENT_AUDIO_CLIP':  112,
    'EVENT_AUDIO_CONVO':  113,
    'EVENT_SCORE':  21,
    'EVENT_APP_SERVICE':  23,
    'EVENT_SLEEP_TRUTH':  20,
    'EVENT_CHARGE':  101,
    'EVENT_CHARGE_DUARATION':  17,
    'EVENT_ACCELEROMETER':  200,
    'EVENT_ACTIVITY':  201,
    'EVENT_DEBUG':  999
}

andKeys = pd.DataFrame(list(andKeys.items()), columns=['Label', 'Key'])

andf = pd.read_csv('/home/jmocel/trelium/MULTICAST-ps/src/data/andtypes.csv')
andf = andf.merge(andKeys,how='left',left_on='data_type',right_on='Key')


In [3]:
andf.to_csv('andtypes_withid.csv')

In [2]:
path = os.path.abspath('/opt/data/multicast/phone')
iosSenses = glob((path)+'/*/*.db')
andSenses = glob((path)+'/*/*.dbr')
to_analyze =  andSenses
#to_analyze.append('ios,/opt/data/multicast/phone/65c6250d55b3c80010090567/65c6250d55b3c80010090567_1710140007_mca.db')


In [18]:
def explode_json(df,ev_id, drop_timestamp = False):
    if drop_timestamp: 
        df1 = df.loc[df['event_id'] == ev_id].drop(['timestamp'],axis = 1)
    else:
        df1 = df.loc[df['event_id'] == ev_id]
    df2 = df.loc[df['event_id'] == ev_id,'data'].apply(json.loads).apply(pd.Series)
    return pd.concat([df1, df2], axis = 1)

def extract_from_json_list(row):
    # Convert the bytes to string
    json_str = row['data'].decode('utf-8')
    # Convert the string to a dictionary
    json_dict = json.loads(json_str)
    # Extract the data list and attach timestamp and user_id
    key = list(json_dict.keys())[0]
    return [
                {**wifi_entry, 'timestamp': row['timestamp'], 'user_id': row['user_id']} 
                for wifi_entry in json_dict[key]
            ]

dbloc = '/opt/data/multicast/phone/65391c7c55b3c800135fd2d2/1699662051794_65391c7c55b3c800135fd2d2.dbr'

conn = sqlite3.connect(dbloc)
conn2 = sqlite3.connect(to_analyze[17])
df = pd.read_sql(sql="SELECT * FROM events", con=conn)
df2 = pd.read_sql(sql="SELECT * FROM events", con=conn2)

df_ex = pd.DataFrame()
df_ex = pd.concat([df2, df]) 
df_ex = df_ex[['timestamp','data_type','data']]
df_ex.rename(columns={
                'data_type' : 'event_id'
                                            }, inplace = True )
df_ex['timestamp'] = pd.to_datetime(df_ex['timestamp'], unit='s', utc=True).dt.tz_convert('Europe/Zurich')
df_ex['user_id'] = dbloc.split(os.sep)[-1].split('_')[1].split('.')[0] #get a portion of the folder name 



In [46]:
#android
def parse_and_df(df_ex, dbloc):
    def explode_json(df,ev_id, drop_timestamp = False):
        if drop_timestamp: 
            df1 = df.loc[df['event_id'] == ev_id].drop(['timestamp'],axis = 1)
        else:
            df1 = df.loc[df['event_id'] == ev_id]
        df2 = df.loc[df['event_id'] == ev_id,'data'].apply(json.loads).apply(pd.Series)   
        return pd.concat([df1, df2], axis = 1)

    def extract_from_json_list(row):
        # Convert the bytes to string
        json_str = row['data'].decode('utf-8')
        # Convert the string to a dictionary
        json_dict = json.loads(json_str)
        # Extract the data list and attach timestamp and user_id
        key = list(json_dict.keys())[0]
        return [
                    {**entry, 'timestamp': row['timestamp'], 'user_id': row['user_id']} 
                    for entry in json_dict[key]
                ]


    df_ex['timestamp'] = pd.to_datetime(df_ex['timestamp'], unit='s', utc=True).dt.tz_convert('Europe/Zurich')
    df_ex['user_id'] = dbloc.split(os.sep)[-1].split('_')[1].split('.')[0] #get a portion of the folder name 
    ret = dict()
    ret['DEVICE_INFO'] = df_ex[['user_id']].copy()
    ret['DEVICE_INFO'] = ret['DEVICE_INFO'].assign(os='android')
    ret['DEVICE_STATE'] = pd.DataFrame()


    for ev_id in df_ex['event_id'].unique():
        #ret = dict()
        if ev_id == 171:    
            dfp = explode_json(df_ex,ev_id, drop_timestamp=False)   
            #dfp['timestamp'] = pd.to_datetime(dfp['timestamp'], unit='s', utc=True).dt.tz_convert('Europe/Zurich')
            dfp.rename(columns={'level' : 'battery_left', "state": 'battery_state'}, inplace = True)
            dfp = dfp.astype({'battery_left': float, "battery_state": int})
            ret['BATTERY_LEVEL'] = dfp[['timestamp', 'user_id','battery_left']]
            ret['BATTERY_STATE'] = dfp[['timestamp', 'user_id','battery_state']]
            
        elif ev_id == 2: 
            dfp = explode_json(df_ex,ev_id)
            dfp.rename(columns={
                                'ALTITUDE' : 'altitude',
                                'LONGITUDE' : 'longitude',
                                'LATITUDE' : 'latitude',
                                'ACCURACY' : 'accuracy',
                                'PROVIDER' : 'source',
                                'STAELLITES' : 'satellites'
                                                }, inplace = True )     
            ret['LOCATION'] = dfp[['timestamp', 'user_id','latitude', 'longitude', 'accuracy', 'altitude','source']]
            desired_columns = {'timestamp', 'user_id', 'satellites', 'SPEED', 
                   'NEWWORKLOCATIONSOURCE', 'BEARING', 'HASBEARING', 
                   'NEWWORKLOCATIONTYPE', 'HASSPEED', 'TRAVELSTATE'}
            
            present_columns = desired_columns & set(dfp.columns)
            missing_columns = desired_columns - set(dfp.columns)
            df_c = dfp[list(present_columns)].copy()
            df_c[list(missing_columns)] = None
            ret['LOCATION_MORE'] = df_c

        elif ev_id == 902: #location ping one single numerical field
            dfp = explode_json(df_ex, ev_id)    
            dfp.rename(columns={0 : 'ping'}, inplace = True )    
            ret['LOCATION_PING'] = dfp[['timestamp', 'user_id','ping']]
        elif ev_id == 91:  
            dfp = explode_json(df_ex,ev_id,drop_timestamp=False)
            dfp.rename(columns={'SSID' : 'ssid'}, inplace = True)    
            ret['WIFI_CONNECTED'] = dfp[['timestamp', 'user_id', 'bssid', 'ssid']]
        elif ev_id == 9: 
            wifi_data_list = df_ex.loc[df_ex['event_id'] == ev_id].apply(extract_from_json_list, axis=1).explode().tolist()
            wifi_data = pd.json_normalize(wifi_data_list)
            if not wifi_data.empty:
                #wifi_data['timestamp'] = pd.to_datetime(wifi_data['timestamp'], unit='s', utc=True).dt.tz_convert('Europe/Zurich')
                wifi_data.columns = [x.lower() for x in wifi_data.columns]
                ret['WIFI_SCANNED'] = wifi_data
            else:
                par  = df_ex.loc[df_ex['event_id'] == ev_id, ['timestamp', 'user_id']].copy()
                par[['bssid', 'ssid']] = None
                ret['WIFI_SCANNED'] = par
        elif ev_id == 10:   
            devices_data_list = df_ex.loc[df_ex['event_id'] == ev_id].apply(extract_from_json_list, axis=1).explode().tolist()
            devices_df = pd.json_normalize(devices_data_list)
            #devices_df['timestamp'] = pd.to_datetime(devices_df['timestamp'], unit='s', utc=True).dt.tz_convert('Europe/Zurich')
            if not devices_df.empty:
                devices_df.rename(columns={'DEVICE' : 'bt_address', 'RSSI' : 'bt_rssi', 'CLASS' : 'bt_class'}, inplace = True)
                ret['BLUETOOTH'] = devices_df[['timestamp', 'user_id', 'bt_address', 'bt_rssi','bt_class']]
            else:
                par = df_ex.loc[df_ex['event_id'] == ev_id, ['timestamp','user_id']].copy()
                par[['bt_address', 'bt_rssi','bt_class']] = None
                ret['BLUETOOTH'] = par
             
        elif ev_id ==  202:       #TODO? also include steps since boot?
            dfp = explode_json(df_ex,ev_id, drop_timestamp = True)
            dfp['start_time'] = pd.to_datetime(dfp['start_time'], unit='ms', utc=True).dt.tz_convert('Europe/Zurich') 
            dfp['end_time'] = pd.to_datetime(dfp['end_time'], unit='ms', utc=True).dt.tz_convert('Europe/Zurich')        
            ret['STEPS'] = dfp[['start_time', 'end_time','user_id', 'steps']]
        elif ev_id == 210:   #"[{'number': '83653d9d0e8628eb301cef41df5722502f50eb94', 'type': 2, 'date': 1701967333263, 'duration': 73}]"
            ll = df_ex.loc[df_ex['event_id'] == 210,'data'].iloc[0].decode('utf-8')
            ll = ast.literal_eval(ll)
            #restructure the data field so that it matches the wifi data format, to use already built functions
            df_ex.loc[df_ex['event_id'] == 210, 'data'] = json.dumps({'calls' : ll}).encode('utf-8')
            calls_data_list = df_ex.loc[df_ex['event_id'] == 210].apply(extract_from_json_list, axis=1).explode().tolist()
            calls_data = pd.json_normalize(calls_data_list)
            calls_data.drop(['timestamp'],axis = 1, inplace=True) #drop timestamp relative to data dump
            calls_data.rename(columns={'date' : 'timestamp', 'type':'callType'}, inplace = True) #keep only internal timestamp
            calls_data = calls_data.astype({"duration": float})
            ret['CALL_LOG'] = calls_data[['timestamp', 'user_id', 'number', 'callType', 'duration']] #TODO harmonize with ios
        elif ev_id == 136:    
            dfp = explode_json(df_ex,ev_id)   
            dfp.rename(columns={
                                'screen_state' : 'LockState'}, inplace = True )
            dfp = dfp.astype({'LockState': float})
            dfp = dfp.astype({'LockState': bool})
            ret['SCREEN'] = dfp[['timestamp', 'user_id','LockState']]
        elif ev_id == 211:     #"[{'address': 'a41b5e08e76671e7507e7e83e69d71305ad163e8', 'type': 1, 'date': 1699082630522, 'read': 1, 'body': 18, 'status': -1, 'thread_id': 6}]",
            ll = df_ex.loc[df_ex['event_id'] == 211,'data'].iloc[0].decode('utf-8')
            ll = ast.literal_eval(ll)
            #restructure the data field so that it matches the wifi data format, to use already built functions
            df_ex.loc[df_ex['event_id'] == 211, 'data'] = json.dumps({'calls' : ll}).encode('utf-8')
            sms_data_list = df_ex.loc[df_ex['event_id'] == 211].apply(extract_from_json_list, axis=1).explode().tolist()
            sms_data = pd.json_normalize(sms_data_list)
            sms_data.drop(['timestamp'],axis = 1, inplace=True)
            sms_data.rename(columns={'date' : 'timestamp'}, inplace = True)
            sms_data['timestamp'] = pd.to_datetime(sms_data['timestamp'], unit = 'ms', utc=True).dt.tz_convert('Europe/Zurich') 
            ret['SMS'] = sms_data
        elif ev_id == 22:  #timestamp	time_in_foreground	package_name	package_category	user_id
            apps_data_list = df_ex.loc[df_ex['event_id'] == 22].apply(extract_from_json_list, axis=1).explode().tolist()
            apps_df = pd.json_normalize(apps_data_list)
            apps_df.drop(['timestamp'],axis = 1, inplace=True) #drop timestamp relative to data dump
            apps_df['last_time_used'] = pd.to_datetime(apps_df['last_time_used'], unit='ms', utc=True).dt.tz_convert('Europe/Zurich')
            apps_df.rename(columns={'last_time_used' : 'timestamp'}, inplace = True)
            ret['APP_USAGE'] = apps_df #TODO look into complete dataset to see all categories present in the data and remap to numbers "package_category" column 
        elif ev_id == 11: 
            ret['SERVICES_STARTED'] = [] 
        elif ev_id == 199:  
            ret['SERVICES_RUNNING'] = [] 
        elif ev_id == 301:  
            dfp = explode_json(df_ex,301, drop_timestamp=False)  
            dfp.drop(['data'],axis = 1, inplace=True)
            ret['NOTIFICATIONS'] = dfp

    ret['DEVICE_INFO'] = ret['DEVICE_INFO'].drop_duplicates()
    return ret

## IOESSE

In [5]:
conn = sqlite3.connect(to_analyze[15])
conn2 = sqlite3.connect(to_analyze[17])
conn_act = sqlite3.connect('/opt/data/multicast/phone/65c6250d55b3c80010090567/65c6250d55b3c80010090567_1709568919_mca.db')

df_act = pd.read_sql(sql="SELECT * FROM events", con=conn_act)
df = pd.read_sql(sql="SELECT * FROM events", con=conn)
df2 = pd.read_sql(sql="SELECT * FROM events", con=conn2)

df3 = pd.DataFrame()

df3 = pd.concat([df3,df2, df_act]) 
#df3['os'] = 'android'
#df3
df3.rename(columns={
                            'event_data' : 'data',
                            'event_time' : 'timestamp'
                                            }, inplace = True )


#df3= df3.astype({"timestamp": float})

df3['timestamp'] = pd.to_datetime(df3['timestamp'], unit='s', utc=True).dt.tz_convert('Europe/Zurich')
#df3['xdxd'] = df3['timestamp'].dt.strftime('yyyy-MM-dd HH:mm:ss') #XSD datetimestamp


In [24]:
dfp = pd.concat([df3.loc[df3['event_id'] == 987],df3.loc[df3['event_id'] == 987,'data'].apply(json.loads).apply(pd.Series)], axis=1)

In [6]:
df3

,user_id,device_id,timestamp,event_id,data
0,65772834d2ad47000e9adc7a,e932b93e-c2ef-4576-b8f0-3abf559f78da,2023-12-14 01:14:05+01:00,987,b'1'
1,65772834d2ad47000e9adc7a,e932b93e-c2ef-4576-b8f0-3abf559f78da,2023-12-14 01:14:35+01:00,987,b'1'
2,65772834d2ad47000e9adc7a,e932b93e-c2ef-4576-b8f0-3abf559f78da,2023-12-14 01:15:05+01:00,151,"b'{\n ""latitude"" : ""47.411208"",\n ""longitude..."
3,65772834d2ad47000e9adc7a,e932b93e-c2ef-4576-b8f0-3abf559f78da,2023-12-14 01:15:05+01:00,987,b'1'
4,65772834d2ad47000e9adc7a,e932b93e-c2ef-4576-b8f0-3abf559f78da,2023-12-14 01:15:35+01:00,987,b'1'
...,...,...,...,...,...
219,65c6250d55b3c80010090567,fcf23cdb-e788-47a6-bc49-9d4881387ae4,2024-03-04 17:14:48+01:00,14,"b'{\n ""LockState"" : ""0""\n}'"
220,65c6250d55b3c80010090567,fcf23cdb-e788-47a6-bc49-9d4881387ae4,2024-03-04 17:14:49+01:00,13,"b'{\n ""brightness"" : ""0.700000"",\n ""timestam..."
221,65c6250d55b3c80010090567,fcf23cdb-e788-47a6-bc49-9d4881387ae4,2024-03-04 17:14:54+01:00,13,"b'{\n ""brightness"" : ""0.000000"",\n ""timestam..."
222,65c6250d55b3c80010090567,fcf23cdb-e788-47a6-bc49-9d4881387ae4,2024-03-04 17:14:54+01:00,14,"b'{\n ""LockState"" : ""1""\n}'"


In [70]:
#df_ex = df3.join(df3['data'].apply(json.loads).apply(pd.Series)) #use concat instead to explode json 
#pd.concat([df3.loc[df3['event_id'] == 18],pd.json_normalize(df3.loc[df3['event_id'] == 18,'data'], max_level=1)], axis=1) 
df_ex = pd.concat([df3.loc[df3['event_id'] == 18],df3.loc[df3['event_id'] == 18,'data'].apply(json.loads).apply(pd.Series)], axis=1)

In [9]:
'824 count'[:-6]

'824'

In [50]:
def parse_ios_df(df_ex):
    """Take as input one pandas dataframe for ios sensing streams and 
    return a dictionary with keys corresponding to the sensing stream ids contained in the input dataframe 
    and pandas dataframes of the desired format as values """
    
    #TODO input values might contain quotation marks, semicolons, % and _ wildcard characters
    #TODO check in the final device info table whether there are multiple device ids connected to the same user_id. 
    # If so it makes more sense to save data in sensor tables by device id and not user id, for ios. 
    
    df_ex['timestamp'] = pd.to_datetime(df_ex['timestamp'], unit='s', utc=True).dt.tz_convert('Europe/Zurich')
    ret = dict()
    ret['DEVICE_INFO'] = df_ex[['user_id','device_id']].copy()
    ret['DEVICE_INFO'] = ret['DEVICE_INFO'].assign(os='ios')
    ret['DEVICE_STATE'] = pd.DataFrame()

    def explode_json(df,ev_id, drop_timestamp = False):
        if drop_timestamp: 
            df1 = df.loc[df['event_id'] == ev_id].drop(['timestamp'],axis = 1)
        else:
            df1 = df.loc[df['event_id'] == ev_id]
        try:    
            df2 = df.loc[df['event_id'] == ev_id,'data'].apply(json.loads).apply(pd.Series)
        except JSONDecodeError:
            df2 = df.loc[df['event_id'] == ev_id,'data'].apply(lambda x: x.decode('utf-8'))
            df2 = df2.apply(lambda x: x + '}' if not x.strip().endswith('}') else x)
            df2 = df2.apply(lambda x: x + '"}' if not x.strip().endswith('"}') else x)
            df2 = df2.apply(json.loads).apply(pd.Series)    
        
        return pd.concat([df1, df2], axis = 1)

    for ev_id in df_ex['event_id'].unique():
        #ret = dict()
        if ev_id == 151: 
            dfp = explode_json(df_ex,ev_id)
            dfp = dfp[['timestamp', 'user_id', 'latitude', 'longitude', 'accuracy', 'altitude']]
            dfp['source'] = 'all'
            try:
                pd.concat([ret['LOCATION'],dfp])
            except KeyError:
                ret['LOCATION'] = dfp
        elif ev_id == 152:
            dfp = explode_json(df_ex,ev_id)        
            dfp = dfp[['timestamp', 'user_id','latitude', 'longitude', 'accuracy', 'altitude']]
            dfp['source'] = 'app'
            try:
                pd.concat([ret['LOCATION'],dfp])
            except KeyError:
                ret['LOCATION'] = dfp
        elif ev_id == 18: 
            dfp = explode_json(df_ex,ev_id,drop_timestamp=True)
            dfp['timestamp'] = pd.to_datetime(dfp['timestamp'], unit='s', utc=True).dt.tz_convert('Europe/Zurich')
            ret['WIFI_CONNECTED'] = dfp[['timestamp', 'user_id', 'bssid', 'ssid']]
        elif ev_id == 181: 
            dfp = explode_json(df_ex,ev_id,drop_timestamp=True)
            dfp['timestamp'] = pd.to_datetime(dfp['timestamp'], unit='s', utc=True).dt.tz_convert('Europe/Zurich')        
            #ret['DEVICE_STATE'] = pd.concat([ret['DEVICE_STATE'], dfp[['start_date', 'end_date', 'user_id', 'wifi_connected_old', 'wifi_connected', 'wifi_enabled']] ], axis = 1)
            ret['WIFI_STATE'] = dfp[['timestamp', 'user_id', 'wifi_connected', 'wifi_enabled']].drop_duplicates()
        elif ev_id == 19:   
            dfp = explode_json(df_ex,ev_id,drop_timestamp=True)
            dfp['timestamp'] = pd.to_datetime(dfp['timestamp'], unit='s', utc=True).dt.tz_convert('Europe/Zurich')
            ret['BLUETOOTH'] = dfp[['timestamp', 'user_id', 'bt_address', 'bt_rssi', 'bt_name']] 
        elif ev_id ==  21:              # "1709500823,15,9.089999999850988,0,0 #will keep the timestamp present in the 'data' field 
            dfp = df_ex.loc[df_ex['event_id'] == ev_id, ['user_id', 'data']]
            dfp['data'] = dfp['data'].str.decode("utf-8")
            dfp = pd.concat([dfp['user_id'], dfp['data'].str.split(',', expand=True)], axis=1)
            dfp.rename(columns={
                                0 : 'timestamp',
                                1 : 'field1',
                                2 : 'field2',
                                3 : 'field3',
                                4 : 'field4'
                                                }, inplace = True ) 
            dfp['timestamp'] = pd.to_datetime(dfp['timestamp'], unit='s', utc=True).dt.tz_convert('Europe/Zurich')        
            ret['STEPS_IOS'] = dfp
        elif ev_id == 22:   
            dfp = explode_json(df_ex,ev_id)
            dfp = dfp.loc[dfp['sample_type'] == 'HKQuantityTypeIdentifierStepCount']
            dfp['sample_quantity'] = [x[:-6] for x in dfp['sample_quantity']]
            dfp['start_date'] = pd.to_datetime(dfp['start_date'], infer_datetime_format=True).dt.tz_convert('Europe/Zurich')
            dfp['end_date'] = pd.to_datetime(dfp['end_date'], infer_datetime_format=True).dt.tz_convert('Europe/Zurich')
            dfp.rename(columns={'end_date' : 'end_time',
                                'start_date' : 'start_time',
                                'sample_quantity' : 'steps'}, inplace=True)

            ret['STEPS'] = dfp[['start_time', 'end_time','user_id', 'steps']]

            pattern = r'name:(.*?), bundle:(.*?), version:(.*?), productType:(.*?), operatingSystemVersion:(.*?)>'
            dfp[['name', 'bundle', 'version', 'productType', 'operatingSystemVersion']] = dfp['source'].str.extract(pattern)

            #dfp['operatingSystemVersion'] = dfp['source'].str.split("""\n""", expand=True)[2].str.split("""=""", expand=True)[1].str.strip('"').str.strip(';')
            #dfp['productType'] = dfp['source'].str.split("""\n""", expand=True)[3].str.split("""=""", expand=True)[1].str.strip('"').str.strip(';')
            #ret['DEVICE_STATE'] = pd.concat([ret['DEVICE_STATE'], dfp[['user_id','start_date','end_date','operatingSystemVersion']]], axis = 0)
            #ret['DEVICE_INFO'].loc[:,'productType'] = dfp['productType'].iloc[0]

            ret['DEVICE_INFO'] = dfp[['name', 'bundle', 'version', 'productType', 'operatingSystemVersion','start_time']]
        elif ev_id == 23:   
            dfp = explode_json(df_ex,ev_id,drop_timestamp=True)   
            dfp = dfp.astype({"duration": float})
            dfp['timestamp'] = pd.to_datetime(dfp['timestamp'], unit='s', utc=True).dt.tz_convert('Europe/Zurich')
            ret['CALL_LOG'] = dfp[['timestamp', 'user_id', 'callId', 'callType', 'duration']]
        elif ev_id == 987:     
            dfp = explode_json(df_ex,ev_id)
            dfp.rename(columns={
                                0 : 'heart_beat'}, inplace=True)
            ret['HEART_BEAT'] = dfp[['timestamp', 'user_id','heart_beat']]  
        elif ev_id == 16:  
            dfp = df_ex.loc[df_ex['event_id'] == ev_id, ['user_id', 'timestamp', 'data']]
            dfp['data'] = dfp['data'].str.decode("utf-8")
            dfp = pd.concat([dfp[['timestamp','user_id']], dfp['data'].str.split(',', expand=True)], axis=1)
            dfp.rename(columns={
                                0 : 'activity',
                                1 : 'confidence'
                                                }, inplace = True )
            ret['ACTIVITY'] = dfp[['timestamp','user_id','activity', 'confidence']]
        elif ev_id == 13:  
            dfp = explode_json(df_ex,ev_id,drop_timestamp=True)   
            dfp = dfp.astype({"brightness": float})
            dfp['timestamp'] = pd.to_datetime(dfp['timestamp'], unit='s', utc=True).dt.tz_convert('Europe/Zurich')
            ret['BRIGHTNESS'] = dfp[['timestamp', 'user_id', 'brightness']]
        
        elif ev_id == 14:     #anche questo è state
            dfp = explode_json(df_ex,ev_id)   
            dfp = dfp.astype({'LockState': float})
            dfp = dfp.astype({'LockState': bool})
            ret['SCREEN'] = dfp[['timestamp', 'user_id','LockState']]

        elif ev_id == 111: #battery state TODO based on if this data comes in at every state change, complete it by setting end_date as the time of the following update 
            dfp = explode_json(df_ex,ev_id, drop_timestamp=True) 
            dfp['timestamp'] = pd.to_datetime(dfp['timestamp'], unit='s', utc=True).dt.tz_convert('Europe/Zurich')
            #dfp.rename(columns={'timestamp' : 'start_date'}, inplace = True)
            dfp = dfp.astype({"battery_state": int})
            #ret['DEVICE_STATE'] = pd.concat([ret['DEVICE_STATE'], dfp[['user_id','start_date','battery_state']]])
            ret['BATTERY_STATE'] = dfp[['user_id','timestamp','battery_state']]

        elif ev_id == 11:     #battery 
            dfp = explode_json(df_ex,ev_id, drop_timestamp=True)   
            dfp['timestamp'] = pd.to_datetime(dfp['timestamp'], unit='s', utc=True).dt.tz_convert('Europe/Zurich')
            dfp = dfp.astype({'battery_left': float})
            ret['BATTERY_LEVEL'] = dfp[['timestamp', 'user_id','battery_left']]
        

    ret['DEVICE_INFO'] = ret['DEVICE_INFO'].drop_duplicates()
    ret['DEVICE_INFO'].rename(columns={ 'productType' : 'product_type'}, inplace=True)

        
    return ret
        

## Testing parsing functions for android and ios

In [ ]:
#testing the parsing on all example tables 
path = os.path.abspath('/opt/data/multicast/phone')
iosSenses = glob((path)+'/*/*.db')
andSenses = glob((path)+'/*/*.dbr')
to_analyze =  iosSenses
error_files = set()

df = pd.DataFrame()

for dbloc in tqdm(to_analyze):
    conn = sqlite3.connect(dbloc)
    try:
        dft = pd.read_sql(sql="SELECT * FROM events", con=conn)
    except pd.errors.DatabaseError:
        ex_type, ex_value, ex_traceback = sys.exc_info()
        if str(ex_value) == """Execution failed on sql 'SELECT * FROM events': no such table: events""":
            pass
        else:
            #logging.error(f"Can't read database file: {dbloc}. --- error: {ex_value} ")
            print(f"Can't read database file: {dbloc}. --- error: {ex_value} ")
            error_files.add(dbloc)
        continue
    
    print(dbloc)
    if dbloc[-3:] == 'dbr':
        #dft = dft.merge(and_map_df,how='left',left_on='data_type',right_on='Key')
        #dft = dft[['timestamp','data_type','data','label']]
        dft = dft[['timestamp','data_type','data']]
        dft.rename(columns={
                            'data_type' : 'event_id'
                                            }, inplace = True )

        _ = parse_and_df(dft, dbloc)
        
    elif dbloc[-3:] == '.db':
        #dft = dft.merge(ios_map_df,how='left',left_on='event_id',right_on='Key')
        dft = dft[['event_time','event_id','event_data','device_id', 'user_id']]
        #dft['event_data'] = dft['event_data'].map(decode_json) 
        #dft['os'] = 'ios'
        dft.rename(columns={
                            'event_data' : 'data',
                            'event_time' : 'timestamp'
                                            }, inplace = True )
        _ = parse_ios_df(dft)
            

#best to run it as module and log errors along with filepaths 


In [41]:


dbloc = "/opt/data/multicast/phone/65ba3fe655b3c8001008faf1/1706854401199_65ba3fe655b3c8001008faf1.dbr"
conn = sqlite3.connect(dbloc)
df_ex = pd.read_sql(sql="SELECT * FROM events", con=conn)

df_ex = df_ex[['timestamp','data_type','data']]
df_ex.rename(columns={
                'data_type' : 'event_id'
                                            }, inplace = True )
#df_ex['timestamp'] = pd.to_datetime(df_ex['timestamp'], unit='s', utc=True).dt.tz_convert('Europe/Zurich')
#df_ex['user_id'] = dbloc.split(os.sep)[-1].split('_')[1].split('.')[0] #get a portion of the folder name 


In [42]:
riecc = parse_and_df(df_ex, dbloc)


In [44]:
riecc['BLUETOOTH']

,timestamp,user_id,bt_address,bt_rssi,bt_class
321,2024-02-02 07:59:05.494999808+01:00,65ba3fe655b3c8001008faf1,None,None,None


In [34]:
def explode_json(df,ev_id, drop_timestamp = False):
    if drop_timestamp: 
        df1 = df.loc[df['event_id'] == ev_id].drop(['timestamp'],axis = 1)
    else:
        df1 = df.loc[df['event_id'] == ev_id]
    df2 = df.loc[df['event_id'] == ev_id,'data'].apply(json.loads).apply(pd.Series)   
    return pd.concat([df1, df2], axis = 1)

def extract_from_json_list(row):
    # Convert the bytes to string
    json_str = row['data'].decode('utf-8')
    # Convert the string to a dictionary
    json_dict = json.loads(json_str)
    # Extract the data list and attach timestamp and user_id
    key = list(json_dict.keys())[0]
    return [
                {**entry, 'timestamp': row['timestamp'], 'user_id': row['user_id']} 
                for entry in json_dict[key]
            ]


devices_data_list = df_ex.loc[df_ex['event_id'] == 10].apply(extract_from_json_list, axis=1).explode().tolist()
devices_df = pd.json_normalize(devices_data_list)


In [39]:
par = df_ex.loc[df_ex['event_id'] == 10,['timestamp','user_id']]
par[['bt_address', 'bt_rssi','bt_class']] = None
par

,timestamp,user_id,bt_address,bt_rssi,bt_class
321,2024-02-02 07:59:05.494999808+01:00,65ba3fe655b3c8001008faf1,None,None,None


In [52]:
path = os.path.abspath('/opt/data/multicast/phone')
iosSenses = glob((path)+'/*/*.db')
andSenses = glob((path)+'/*/*.dbr')
to_analyze = andSenses
len(iosSenses)

28720

In [57]:
dbloc = '/opt/data/multicast/phone/65342ffbb573c8000e86ad06/1697978054177_65342ffbb573c8000e86ad06.dbr'


conn = sqlite3.connect(dbloc)
conn2 = sqlite3.connect(to_analyze[17])
df = pd.read_sql(sql="SELECT * FROM events", con=conn)
df2 = pd.read_sql(sql="SELECT * FROM events", con=conn2)

df_ex = pd.DataFrame()
df_ex = pd.concat([df2, df]) 
df_ex = df_ex[['timestamp','data_type','data']]
df_ex.rename(columns={
                'data_type' : 'event_id'
                                            }, inplace = True )

In [56]:
dfff = parse_ios_df(df)
pprint(dfff.keys()
       )



dict_keys(['DEVICE_INFO', 'DEVICE_STATE', 181, 18, 987, 151, 13, 11, 16, 21, 22, 14, 19, 23, 111])


In [73]:
to_analyze[15].split(os.sep)[-1].split('_')[1].split('.')[0]

'65536c9ed2ad47000e9a8f01'

In [ ]:
to_analyze[0][-3:]


'.db'

In [ ]:
if to_analyze[9999][-3:] == 'dbr':
        print(to_analyze[0])
elif to_analyze[0][-3:] == '.db':
        print(to_analyze[0])


/opt/data/multicast/phone/644604239026c6000edaf51e/644604239026c6000edaf51e_1682310214_mca.db


In [ ]:
dfex = dft.explode('event_data').reset_index(drop=True)
df_out = dfex.join(dfex['event_data'].apply(pd.Series)).drop('event_data', axis=1)
df_out.head()

,user_id,device_id,event_time,event_id,0
0,65490391d2ad47000e9a862b,30c91a6a-e90d-44cb-9047-f0adb49f8674,1699386547,181,wifi_connected_old
1,65490391d2ad47000e9a862b,30c91a6a-e90d-44cb-9047-f0adb49f8674,1699386547,181,timestamp
2,65490391d2ad47000e9a862b,30c91a6a-e90d-44cb-9047-f0adb49f8674,1699386547,181,wifi_enabled
3,65490391d2ad47000e9a862b,30c91a6a-e90d-44cb-9047-f0adb49f8674,1699386547,181,wifi_connected
4,65490391d2ad47000e9a862b,30c91a6a-e90d-44cb-9047-f0adb49f8674,1699386547,18,frequency


### Code to generate csv with datatypes for android

In [ ]:
import logging 
import pandas as pd
from pathlib import Path
from glob import glob 
import os
from tqdm import tqdm 
import sqlite3
import argparse
import json
import sys
from event_ids import iosKeys, andKeys

path = os.path.abspath('/opt/data/multicast/phone')
iosSenses = glob((path)+'/*/*.db')
andSenses = glob((path)+'/*/*.dbr')
to_analyze = andSenses 

#andKeys = pd.DataFrame(list(andKeys.items()), columns=['Label', 'Key'])
error_files = set()
c= 0 
df = pd.DataFrame()

def decode_json(x):
    if isinstance(x, bytes):
        try:
            return json.loads(x.decode())
        except json.JSONDecodeError: 
            return x.decode()
    else:
        return x

for dbloc in tqdm(to_analyze):
    conn = sqlite3.connect(dbloc)
    try:
        dft = pd.read_sql(sql="SELECT * FROM events", con=conn)
    except pd.errors.DatabaseError:
        ex_type, ex_value, ex_traceback = sys.exc_info()
        if str(ex_value) == """Execution failed on sql 'SELECT * FROM events': no such table: events""":
            pass
        else:
            logging.error(f"Can't read database file: {dbloc}. --- error: {ex_value} ")
            error_files.add(dbloc)

    if dbloc[-3:] == 'dbr':
        dft['data'] = dft['data'].map(decode_json)  
        dft['os'] = 'android'
    elif dbloc[-3:] == '.db':
        dft['event_data'] = dft['event_data'].map(decode_json) 
        dft['os'] = 'ios'
        dft.rename(columns={
                            'event_id' : 'data_type',
                            'event_data' : 'data',
                            'event_time' : 'timestamp'
                                            }, inplace = True )

    dft['path'] = dbloc
    df = pd.concat([df,dft]) #TODO launch parse 
    c+=1

        
    conn.close()
    #if c==3000:
    #    break

dfios = df.loc[df['os'] == 'android'].drop_duplicates(subset = 'data_type', keep='last')
dfios.to_csv('andtypes.csv')